In [72]:
# -*- coding: utf-8 -*-

# Run this code if you get errors with missing requirements
import sys
!{sys.executable} -m pip install mysql-connector
import pandas as pd
import numpy as np
import mysql.connector
mydb = mysql.connector.connect(
  host="iiwari-mariadb-server",
  user="root",
  passwd="d41k4Duu",
  database="iiwari_org")

In [ ]:
import matplotlib.pyplot as plt

# Check database contents
mycursor = mydb.cursor(dictionary=True)
mycursor.execute("SELECT node_id, timestamp, x,y,z,q FROM SensorData WHERE (node_id = 3200) AND timestamp > '2019-07-07 13:05:08.078133+00:00' AND timestamp < '2019-07-08 13:05:08.078133+00:00'  AND x > 0  AND x < 10406 AND y < 5220 AND y > 0 AND z = 0"); # 3200
# 
# SELECT node_id, timestamp, x,y,z,q FROM SensorData WHERE node_id = 3224 AND x > 0  AND x < 10406 AND y < 5220 AND y > 0 LIMIT 100
df = pd.DataFrame(mycursor.fetchall())

#for i in df:
    #df.drop(df[df.x > 10406].index, inplace=True)
    #df.drop(df[df.y > 5220].index, inplace=True)
import os
print(os.listdir())

df['timestamp'] = df['timestamp'].astype(str)
df['timestamp'] = df['timestamp'].str.slice(2, -7)
df['timestamp'] = df['timestamp'].astype('datetime64[s]')
print(df['timestamp'].dt.second.head())

print(df.timestamp[0])

a = 0

df2 = df.copy(deep=False)

for i in range (df.timestamp.size-1):
    ajan_erotus = np.datetime64(df.timestamp[i+1]) - np.datetime64(df.timestamp[i])
    ajan_erotus.item().total_seconds()
    ajan_erotus = ajan_erotus / np.timedelta64(1, 's')
    if ajan_erotus == 0.0:
        continue
    x_erotus = df.x[i+1]-df.x[i]
    y_erotus = df.y[i+1]-df.y[i]
    if (x_erotus / ajan_erotus > 1.5 and y_erotus / ajan_erotus > 1.5):
        df2.drop(df.index[i], axis = 0, inplace=True)
        a += 1

print("poistettu liian nopean liikkeen takia "+str(a)+"/"+str(df.timestamp.size-1)+" pistettä")
print(df2.timestamp.size)

        
prev = 0
val = 0

def calc_velocity(time_start, time_end):
    #diff_time = time_start - time_end
    diff_time = np.datetime64(time_start) - np.datetime64(time_end)
    diff_time.item().total_seconds()
    diff_time = diff_time / np.timedelta64(1, 's')
    if(diff_time > 0.1):
        return diff_time
    else:
        return 1
x = 0
for i in range(len(df.x)):
   # print(abs(df.iloc[i+1, 3] - abs(df.iloc[i, 3])))
    #print(abs(df.index[i+1]) - abs(df.index[i]))
    if(i < len(df.x)):
        #print(i , " " , x, " ", len(df.x))
        value1 = calc_velocity(df.iloc[i-x, 1], df.iloc[i-(1+x), 1])
        value2 = int((abs(df.iloc[i-x, 2])) - prev)
        val =  value2 / value1
        if (val > 60 or value2 > 100):
            df.drop([df.index[i-x]], axis = 0, inplace = True)
            prev = abs(df.iloc[i-x, 2])
            x +=1
        else:
            prev = abs(df.iloc[i-x, 2])
x = 0
for i in range(len(df.y)):
   # print(abs(df.iloc[i+1, 3] - abs(df.iloc[i, 3])))
    #print(abs(df.index[i+1]) - abs(df.index[i]))
    if(i < len(df.y)):
        value1 = calc_velocity(df.iloc[i-x, 1], df.iloc[i-(1+x), 1])
        value2 = int((abs(df.iloc[i-x, 3]))-prev)
        val = value2 / value1    
        if (val > 60 or value2 > 100):
            df.drop([df.index[i-x]], axis = 0)
            prev = abs(df.iloc[i-x , 3])
            x +=1
        else:
            prev = abs(df.iloc[i-x, 3])
    #print(prev)
    #if (abs(df.iloc[i+1, 3] - abs(df.iloc[i, 3]))) > 400:
        #print("We int")
        
print(len(df.x))
#devx = [df.iloc[i + 1, 2] - df.iloc[i, 2] for i in range(len(df.x)-1)]
devx = []
for i in range(len(df.x)):
    value1 = calc_velocity(df.iloc[i, 1], df.iloc[i-1, 1])
    val = int((abs(df.iloc[i, 2]) - prev)) / value1
    #print(val)
    devx.append(val)
    prev = abs(df.iloc[i, 2])

xmean = np.mean(devx, axis=0)
xsd = np.std(devx, axis=0)
print(xmean)
print(xsd)
i = 0
final_list = [x for x in df.x if (abs(x) > xmean - 0.5 * xsd)]
#final_list = [x for x in final_list if (x < xmean + 0. * xsd)]
for x in final_list:
    if(x < xmean + 6.75 * xsd):
        df.drop([df.index[i]], axis = 0, inplace = True)
        i -= 1
    i += 1
i = 0
#devy = [df.iloc[i + 1, 3] - df.iloc[i, 3] for i in range(len(df.y)-1)]
devy = []
for i in range(len(df.y)):
    value1 = calc_velocity(df.iloc[i, 1], df.iloc[i-1, 1])
    val = int((abs(df.iloc[i, 3]) - prev)) / value1
    #print(val)
    devy.append(val)
    prev = abs(df.iloc[i, 3])
ymean = np.mean(devy, axis=0)
ysd = np.std(devy, axis=0)
print(ymean)
print(ysd)
final_list1 = [y for y in df.y if (abs(y) > ymean - 0.5 * ysd)]
#final_list = [y for y in final_list if (y < ymean + 0.2 * ysd)]
i = 0
print(len(df.y))
for y in final_list1:
    if(y < ymean + 6.75 * ysd):
        df.drop([df.index[i]], axis = 0, inplace = True)
        i -= 1
    i += 1
    

        
        
#for i in range(df.timestamp.size-1):
#    if(df.x[i] < 1850.0 and df.y[i] > 3000.0):
#        df.drop(df.index[i], axis = 0, inplace = True)

df = df.drop(df[(df.x < 1850.0) & (df.y > 3000.0)].index)

import seaborn as sns
ax = sns.kdeplot(df.x, bw=.15)

import matplotlib.cm as cm
from scipy.ndimage.filters import gaussian_filter


def myplot(x, y, s, bins=1000):
    heatmap, xedges, yedges = np.histogram2d(x, y, bins=bins)
    heatmap = gaussian_filter(heatmap, sigma=s)

    extent = [xedges[-1], xedges[0], yedges[-1], yedges[0]]
    return heatmap.T, extent


fig, axs = plt.subplots(2, 2)

# Generate some test data
x = df.x
y = df.y
sigmas = [0, 16, 32, 64]

for ax, s in zip(axs.flatten(), sigmas):
    if s == 0:
        ax.invert_yaxis()
        ax.plot(x, y, 'k.', markersize=5)
        ax.set_title("Scatter plot")
    else:
        img, extent = myplot(x, y, s)
        img = np.flipud(img)
        #ax.imshow(img, extent=extent, origin='upper', cmap=cm.jet)
        ax.imshow(img, extent=[-800, 10400, 5220, -250], origin='lower', cmap=cm.jet)
        ax.set_title("Smoothing with  $\sigma$ = %d" % s)

plt.show()

plt.clf()
img = plt.imread("pic.png")
plt.rcParams['figure.figsize'] = 10, 5
fig, ax = plt.subplots()
#ax.imshow(img, extent=[0, df.x.max(), 0,df.y.max()])
ax.imshow(img, extent=[-800, 10400, 5400, -200])
#ax.plot(df.x, df.y, '--', linewidth=5, color='firebrick')
ax.plot(df.x, df.y , linewidth=1, color='firebrick')
plt.show()

mycursor = mydb.cursor(dictionary=True)
mycursor.execute("SELECT node_id, COUNT(node_id) FROM SensorData   WHERE timestamp > '2019-05-24 13:05:08.078133+00:00' AND timestamp < '2019-05-29 13:05:08.078133+00:00'  AND x > 0  AND X < 10406 AND y < 5220 AND y > 0 AND   q <100 AND z = 0 GROUP BY node_id") # SELECT DISTINCT node_id FROM SensorData
# SELECT node_id, COUNT(node_id) FROM SensorData   WHERE timestamp > '2019-05-24 13:05:08.078133+00:00' AND timestamp < '2019-05-29 13:05:08.078133+00:00'  AND x > 0  AND X < 10406 AND y < 5220 AND y > 0 AND   q <100 AND z = 0 GROUP BY node_id
nodes = pd.DataFrame(mycursor.fetchall())
print(nodes)

nodes = nodes.rename(columns={'COUNT(node_id)': 'amount'})
objects = nodes.node_id
y_pos = np.arange(len(objects))
performance = nodes.amount

plt.clf()
plt.barh(y_pos, performance, align='center', alpha=0.5)
plt.yticks(y_pos, objects)
plt.xlabel('Määrä')
plt.title('Kärrien käyttömäärä')
plt.show()

plt.clf()
plt.pie(performance, labels=objects, autopct='%1.1f%%')
plt.axis('equal')
plt.title("Kärryjen käyttö eriteltynä prosentteina")
plt = plt.gcf()
plt.set_size_inches(8,8)
plt.show()

['.git', 'pic.png', 'lähdekoodi.txt', 'docker-compose.yml', '.ipynb_checkpoints', 'README.md', 'iiwari.ipynb', 'init.sql']
0    2
1    3
2    3
3    4
4    5
Name: timestamp, dtype: int64
2019-07-07 13:36:02
poistettu liian nopean liikkeen takia 1281/12815 pistettä
11535
12413
-1.686483167940886
38.90440460111482


In [67]:
from collections import Counter
def Dublicates(x):
    elements = dict()
    for elem in elements:
        if elem in elements:
            elements[elem] += 1
        else:
            elements[elem] = 1
 
    elements = { key:value for key, value in elements.items() if value > 1}
    return elements

elements = dict(Counter(df.x))

elements = { key:value for key, value in elements.items() if value > 1}
 
for key, value in elements.items():
    print('x: arvo', key,'esiintyy', value, 'kertaa')

for key, value in elements.items():
    if value > 70:
        value = 1
    print('x: arvo',key,'esiintyy', value, 'kertaa')

x: arvo 1043 esiintyy 2 kertaa
x: arvo 1982 esiintyy 2 kertaa
x: arvo 3208 esiintyy 2 kertaa
x: arvo 5706 esiintyy 2 kertaa
x: arvo 5791 esiintyy 2 kertaa
x: arvo 6713 esiintyy 2 kertaa
x: arvo 7271 esiintyy 2 kertaa
x: arvo 7396 esiintyy 2 kertaa
x: arvo 7474 esiintyy 2 kertaa
x: arvo 7483 esiintyy 2 kertaa
x: arvo 7696 esiintyy 2 kertaa
x: arvo 7449 esiintyy 2 kertaa
x: arvo 7546 esiintyy 2 kertaa
x: arvo 7915 esiintyy 2 kertaa
x: arvo 7933 esiintyy 2 kertaa
x: arvo 7986 esiintyy 2 kertaa
x: arvo 8260 esiintyy 3 kertaa
x: arvo 9436 esiintyy 11 kertaa
x: arvo 9588 esiintyy 2 kertaa
x: arvo 9736 esiintyy 3 kertaa
x: arvo 9915 esiintyy 2 kertaa
x: arvo 9710 esiintyy 2 kertaa
x: arvo 9719 esiintyy 2 kertaa
x: arvo 9728 esiintyy 2 kertaa
x: arvo 9611 esiintyy 2 kertaa
x: arvo 9625 esiintyy 2 kertaa
x: arvo 9449 esiintyy 8 kertaa
x: arvo 9376 esiintyy 2 kertaa
x: arvo 9252 esiintyy 2 kertaa
x: arvo 9221 esiintyy 2 kertaa
x: arvo 8545 esiintyy 18 kertaa
x: arvo 8538 esiintyy 4 kertaa
x: arv

In [68]:
def calc_velocity1(time_start, time_end, ):
    #diff_time = time_start - time_end
    diff_time = np.datetime64(time_start) - np.datetime64(time_end)
    diff_time.item().total_seconds()
    diff_time = diff_time / np.timedelta64(1, 's')
    if(diff_time > 0.1):
        return diff_time
    else:
        return 1
x = 0
import math  
def calculateDistance(x1,y1,x2,y2):  
     dist = math.sqrt((x2 - x1)**2 + (y2 - y1)**2)  
     return dist  
devx1 = []
time = []
dist = []
speed = []
i = 1
for i in range(len(df.x)):
    time.append(calc_velocity1(df.iloc[i, 1], df.iloc[i-1, 1]))
    #val = int((abs(df.iloc[i, 2]) - prev)) / value1
    dist.append(calculateDistance(abs(df.iloc[i, 2]), abs(df.iloc[i, 3]),abs(df.iloc[i-1, 2]),  abs(df.iloc[i-1, 3])))
    #print(val)
speed = []
for i in range(len(dist)):
    speed.append((dist[i] / 161)/time[i])
def cal_average(num):
    sum_num = 0
    for t in num:
        sum_num = sum_num + t           

    avg = sum_num / len(num)
    return avg
print(cal_average(speed))

0.16481287860549854
